<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/classifiers/new_decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from sklearn import tree
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [6]:
from IPython.display import SVG, display, HTML

In [7]:
from google.colab import drive
drive.mount('./content', force_remount=False)

Drive already mounted at ./content; to attempt to forcibly remount, call drive.mount("./content", force_remount=True).


In [8]:
!ls
import os
for i in os.listdir('content/'):
  print(i)

content  sample_data
MyDrive
.shortcut-targets-by-id
.file-revisions-by-id
.Trash


In [12]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv

--2021-05-13 10:01:55--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1071192 (1.0M) [text/plain]
Saving to: ‘coref_df_anns60k.csv.1’

coref_df_anns60k.cs 100%[===================>]   1.02M  --.-KB/s    in 0.05s   

2021-05-13 10:01:56 (21.5 MB/s) - ‘coref_df_anns60k.csv.1’ saved [1071192/1071192]



In [21]:
anns_df = pd.read_csv('coref_df_anns60k.csv', delimiter='\t', error_bad_lines=False, header=None)

b'Skipping line 881: expected 9 fields, saw 14\nSkipping line 882: expected 9 fields, saw 14\nSkipping line 883: expected 9 fields, saw 14\nSkipping line 1212: expected 9 fields, saw 14\nSkipping line 1214: expected 9 fields, saw 14\nSkipping line 1528: expected 9 fields, saw 14\nSkipping line 1529: expected 9 fields, saw 14\nSkipping line 1530: expected 9 fields, saw 14\nSkipping line 1859: expected 9 fields, saw 14\nSkipping line 1861: expected 9 fields, saw 14\nSkipping line 2038: expected 9 fields, saw 14\nSkipping line 2043: expected 9 fields, saw 14\nSkipping line 2054: expected 9 fields, saw 14\nSkipping line 2251: expected 9 fields, saw 14\nSkipping line 2252: expected 9 fields, saw 14\nSkipping line 2353: expected 9 fields, saw 14\nSkipping line 2842: expected 9 fields, saw 14\nSkipping line 2847: expected 9 fields, saw 14\nSkipping line 2858: expected 9 fields, saw 14\nSkipping line 3055: expected 9 fields, saw 14\nSkipping line 3056: expected 9 fields, saw 14\nSkipping line 

In [22]:
anns_df

,0,1,2,3,4,5,6,7,8
0,TOKEN,TOPIC_NUM,POST_NUM,SENT_NUM,IS_ANSWER,HEAD,NER,COREFERENCE_CLUSTER,NaN
1,обсуждаем,1,0,0,0,обсуждаем,0,0,NaN
2,препарат,1,0,0,0,обсуждаем,0,0,NaN
3,рисперидон,1,0,0,0,препарат,мед,рисперидон,NaN
4,рисполепт,1,0,0,0,рисперидон,мед,рисполепт,NaN
...,...,...,...,...,...,...,...,...,...
26174,и,22,68,0,0,пошлины,0,0,NaN
26175,таможенные,22,68,0,0,пошлины,0,0,NaN
26176,пошлины,22,68,0,0,есть,0,0,NaN
26177,как,22,68,0,0,есть,0,0,NaN


In [ ]:
cor_df = pd.read_csv('content/MyDrive/coref_fasttext60k.csv', delimiter=' ')

In [ ]:
cor_df

In [ ]:
cor_df.isnull().sum()

TOKEN                     1
POST_NUM                  1
ANIMACY                   1
CASE                      1
GENDER                    1
PERSON                    1
POS                       1
DEPENDENCY                1
HEAD                      1
HEAD_POS                  1
HEAD_ANIMACY              1
HEAD_CASE                 1
HEAD_GENDER               1
HEAD_POS.1                1
SENT_VECTOR               0
NER                       1
COREFERENCE_CLUSTER       4
Unnamed: 17            1788
Unnamed: 18            1797
dtype: int64

In [ ]:
cor_df = cor_df.iloc[:-5,:]
cor_df_test = cor_df.iloc[-5:-1,:]
cor_df = cor_df.drop(['Unnamed: 17', 'Unnamed: 18'], axis=1)
cor_df_test  = cor_df_test .drop(['Unnamed: 17', 'Unnamed: 18'], axis=1)

In [ ]:
cor_df_test = cor_df_test.reset_index()
cor_df_test = cor_df_test.drop(['index'], axis=1)

In [ ]:
cor_df_test = cor_df_test.drop(['level_0'], axis=1)
cor_df_test

,TOKEN,POST_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_POS,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS.1,SENT_VECTOR,NER,COREFERENCE_CLUSTER
0,что,46.0,-,-,-,-,CONJ,fixed,Так,CONJ,-,-,-,CONJ,0.145365668859865,0,0
1,дак,46.0,-,-,-,-,CONJ,nsubj,будет,VERB,-,-,-,VERB,0.145365668859865,0,даклатасвир
2,скоро,46.0,-,-,-,-,ADVB,advmod,будет,VERB,-,-,-,VERB,0.145365668859865,0,0
3,будет,46.0,-,-,-,3per,VERB,ROOT,будет,VERB,-,-,-,VERB,0.145365668859865,0,0


In [ ]:
cor_df

,TOKEN,POST_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_POS,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS.1,SENT_VECTOR,NER,COREFERENCE_CLUSTER
0,На,10.0,-,-,-,-,PREP,case,рынок,NOUN,inan,accs,masc,NOUN,0.2611904942896217,0,0
1,рынок,10.0,inan,accs,masc,-,NOUN,obl,вернулся,VERB,-,-,masc,VERB,0.2611904942896217,0,0
2,снова,10.0,-,-,-,-,ADVB,advmod,вернулся,VERB,-,-,masc,VERB,0.2611904942896217,0,0
3,вернулся,10.0,-,-,masc,-,VERB,ROOT,вернулся,VERB,-,-,masc,VERB,0.2611904942896217,0,0
4,алимемазин,10.0,inan,nomn,masc,-,NOUN,nsubj,вернулся,VERB,-,-,masc,VERB,0.2611904942896217,B-MED,алимемазин
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1788,что,46.0,-,-,-,-,CONJ,fixed,Так,CONJ,-,-,-,CONJ,0.145365668859865,0,0
1789,дак,46.0,-,-,-,-,CONJ,nsubj,будет,VERB,-,-,-,VERB,0.145365668859865,0,даклатасвир
1790,скоро,46.0,-,-,-,-,ADVB,advmod,будет,VERB,-,-,-,VERB,0.145365668859865,0,0
1791,будет,46.0,-,-,-,3per,VERB,ROOT,будет,VERB,-,-,-,VERB,0.145365668859865,0,0


In [ ]:
X = cor_df.drop(['COREFERENCE_CLUSTER', 'NER', 'TOKEN'], axis=1)
X_test = cor_df_test.drop(['COREFERENCE_CLUSTER', 'NER', 'TOKEN'], axis=1)
labels_dict = {}
test_labels_dict = {}

for i in X.columns:
  X[i], lbl = pd.factorize(X[i])
  labels_dict[i] = lbl

for i in X_test.columns:
  X_test[i], lbl = pd.factorize(X_test[i])
  test_labels_dict[i] = lbl 

y = cor_df['COREFERENCE_CLUSTER']
y, y_label = pd.factorize(y)

y_test = cor_df_test['COREFERENCE_CLUSTER']
y_test, y_test_label = pd.factorize(y_test)

In [ ]:
y

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
len(y)

1793

In [ ]:
label[-1]

'омез'

In [ ]:
uniques

Index(['0', 'алимемазин', 'терален', 'сонапакс ', 'велаксин', 'тералиджен',
       'азалептин', 'ремерон', 'золендроновая кислота-B',
       'золендроновая кислота-I', 'золендроновая кислота', 'сторониц',
       'самарий', 'изотопы', 'HIFU-терапия', 'абраксан', 'гемцитабин',
       'капецитабин', 'капицетабин', 'Инорекан', 'софосбувир', 'даклатасвир',
       'рибавирин', 'велпатасвир', 'ледипасвир', 'омез'],
      dtype='object')

In [ ]:
X

,POST_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_POS,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS.1,SENT_VECTOR
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,0,1,1,1,1,1,1,0,1,0
2,0,0,0,0,0,2,2,1,1,1,1,0,1,0
3,0,0,0,1,0,3,3,1,1,1,1,0,1,0
4,0,1,2,1,0,1,4,1,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1788,39,0,0,0,0,9,12,498,2,1,1,3,2,147
1789,39,0,0,0,0,9,4,310,1,1,1,3,1,147
1790,39,0,0,0,0,2,2,310,1,1,1,3,1,147
1791,39,0,0,0,2,3,3,310,1,1,1,3,1,147


In [ ]:
labels_dict['HEAD_POS']

Index(['NOUN', 'VERB', 'CONJ', 'PRTS', '-', 'PRCL', 'ADJF', 'ADVB', 'PRED',
       'INFN', 'NPRO', 'ADJS', 'COMP', 'INTJ', 'NUMR', 'PREP', 'GRND', 'PRTF'],
      dtype='object')

In [ ]:
clf = tree.DecisionTreeClassifier()
clf.fit(X, y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
clf_ent = tree.DecisionTreeClassifier(criterion='entropy')
clf_ent.fit(X, y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
from graphviz import Source
from sklearn import tree

In [ ]:
# tree.plot_tree(clf)

In [ ]:
# graph = Source(tree.export_graphviz(clf, out_file=None, 
#                                     feature_names=list(X), class_names=uniques, filled=True))

# display(SVG(graph.pipe(format='svg')))

In [ ]:
clf.predict(X_test)

array([0, 0, 0, 0])

In [ ]:
y_test

array([0, 1, 0, 0])

In [ ]:
X_test

,POST_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_POS,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS.1,SENT_VECTOR
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,1,1,0,0,0,1,0
2,0,0,0,0,0,1,2,1,1,0,0,0,1,0
3,0,0,0,0,1,2,3,1,1,0,0,0,1,0
